## Homework - Unit 6

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import seaborn as sns
import wget
# import zipfile

In [3]:
data = wget.download('https://github.com/alexeygrigorev/datasets/raw/refs/heads/master/jamb_exam_results.csv')

In [4]:
df = pd.read_csv(data)
df.head()

,JAMB_Score,Study_Hours_Per_Week,Attendance_Rate,Teacher_Quality,Distance_To_School,School_Type,School_Location,Extra_Tutorials,Access_To_Learning_Materials,Parent_Involvement,IT_Knowledge,Student_ID,Age,Gender,Socioeconomic_Status,Parent_Education_Level,Assignments_Completed
0,192,22,78,4,12.4,Public,Urban,Yes,Yes,High,Medium,1,17,Male,Low,Tertiary,2
1,207,14,88,4,2.7,Public,Rural,No,Yes,High,High,2,15,Male,High,None,1
2,182,29,87,2,9.6,Public,Rural,Yes,Yes,High,Medium,3,20,Female,High,Tertiary,2
3,210,29,99,2,2.6,Public,Urban,No,Yes,Medium,High,4,22,Female,Medium,Tertiary,1
4,199,12,98,3,8.8,Public,Urban,No,Yes,Medium,Medium,5,22,Female,Medium,Tertiary,1


In [7]:
df.columns = df.columns.str.lower().str.replace(' ', '_')

In [17]:
df = df.drop('student_id', axis=1)

In [19]:
df = df.fillna(0)

In [20]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [21]:
df_train_full, df_test = train_test_split(df, test_size=0.2, random_state=1)

In [22]:
df_train, df_val = train_test_split(df_train_full, test_size=0.25, random_state=1)

In [ ]:
dv = DictVectorizer(sparse=True)

### Question 1

Let's train a decision tree regressor to predict the jamb_score variable.
- Train a model with max_depth=1.

Which feature is used for splitting the data?